In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import pandas as pd

os.chdir(os.path.abspath('..'))

In [2]:
from dinov2.data.datasets.treesat import TreeSAT, TreeSATWrapper


base_path = '/data/panopticon/datasets/treesat/'
modalities = ['aerial', 's2-mono', 's1-mono']

treesat = TreeSATWrapper(base_path, modalities)

Data path: /data/panopticon/datasets/treesat/train_filenames.lst


In [3]:
treesat.sensor_name_mapping.values()

dict_values(['treesat-s1', 'treesat-s2', 'treesat-aerial'])

In [ ]:
treesat.chn_ids

{'s1-mono': tensor([-1, -2], dtype=torch.int32),
 's2-mono': tensor([ 492,  559,  664,  832,  704,  740,  782,  864, 1613, 2202,  442,  945],
        dtype=torch.int32),
 'aerial': tensor([840, 650, 560, 450], dtype=torch.int32)}

In [5]:
import random
i = random.randint(0, len(treesat)-1)
sample, labels = treesat[i]
sample.keys()

dict_keys(['imgs', 'chn_ids'])

In [6]:
sample['imgs'][0][0].shape, sample['imgs'][1][0].shape, sample['imgs'][2][0].shape

(torch.Size([4, 304, 304]), torch.Size([2, 6, 6]), torch.Size([12, 6, 6]))

In [ ]:
sample['chn_ids'], labels

([tensor([840, 650, 560, 450], dtype=torch.int32),
  tensor([-1, -2], dtype=torch.int32),
  tensor([ 492,  559,  664,  832,  704,  740,  782,  864, 1613, 2202,  442,  945],
         dtype=torch.int32)],
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]))

In [8]:
sample['imgs'][0][0].min(), sample['imgs'][0][0].max(), sample['imgs'][1][0].min(), sample['imgs'][1][0].max(), sample['imgs'][2][0].min(), sample['imgs'][2][0].max()

(tensor(-1.8589),
 tensor(4.8971),
 tensor(-0.0220),
 tensor(0.1916),
 tensor(-1.1887),
 tensor(0.9105))

# Preprocess TreeSAT AI data files

In [9]:
import h5py
import numpy as np
import torch

def replace_nans_with_mean(batch_of_images):
    # Calculate the mean along the specified axis (axis=(2, 3) for temp, channels, height, width)
    image_means = torch.nanmean(batch_of_images, dim=(2, 3), keepdim=True)

    # Create a mask for NaN values
    nan_mask = torch.isnan(batch_of_images)

    # Use PyTorch broadcasting to replace NaN values with corresponding means
    batch_of_images[nan_mask] = image_means.expand_as(batch_of_images)[nan_mask]

    nan_mask = torch.isnan(image_means).any(dim=1).squeeze()

    # Filter out the time steps where there are NaN values
    batch_of_images = batch_of_images[~nan_mask]

    return batch_of_images

data_dir = "/data/panopticon/datasets/treesat/" #path of data

with open(data_dir + "train_filenames.lst", 'r') as file:
    lst_list = [line.strip() for line in file.readlines()]

with open(data_dir + "val_filenames.lst", 'r') as file:
    lst_list += [line.strip() for line in file.readlines()]

with open(data_dir + "test_filenames.lst", 'r') as file:
    lst_list += [line.strip() for line in file.readlines()]

for file_name in lst_list:
    print("original filename : ",file_name)
    print("filename : ", data_dir + "sentinel-ts/" + '.'.join(file_name.split('.')[:-1]) + ".h5")
    with h5py.File(data_dir + "sentinel/" + '.'.join(file_name.split('.')[:-1]) + ".h5", 'r') as file:
        s1_asc = file["sen-1-asc-data"][:]
        s1_asc = replace_nans_with_mean(torch.tensor(s1_asc))
        file_path = data_dir + "s1-asc/" + '.'.join(file_name.split('.')[:-1]) + ".pth"
        s1_asc = torch.cat([s1_asc, (s1_asc[:, 0, :, :]/(s1_asc[:, 1, :, :] + 1e-8)).unsqueeze(1)], dim = 1)
        if torch.isnan(s1_asc).any().item():
            print(file_name)
        torch.save(s1_asc, file_path)
        s1_des = file["sen-1-des-data"][:]
        s1_des = replace_nans_with_mean(torch.tensor(s1_des))
        s1_des = torch.cat([s1_des, (s1_des[:, 0, :, :]/(s1_des[:, 1, :, :] + 1e-8)).unsqueeze(1)], dim = 1)
        if torch.isnan(s1_des).any().item():
            print("s1-des", file_name)
        file_path = data_dir + "s1-des/" + '.'.join(file_name.split('.')[:-1]) + ".pth"
        torch.save(s1_des, file_path)

original filename :  Abies_alba_1_1005_WEFL_NLF.tif
filename :  /data/panopticon/datasets/treesat/sentinel-ts/Abies_alba_1_1005_WEFL_NLF.h5


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/data/panopticon/datasets/treesat/sentinel/Abies_alba_1_1005_WEFL_NLF.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)